In [1]:
import os,pandas as pd,glob
import time,datetime as dt,sys
import dorianUtils.comUtils as comUtils
import smallPowerDash.smallPower as smallPower
start=time.time()
dumperSmallPower = smallPower.SmallPower_dumper()
beckhoff=dumperSmallPower.devices['beckhoff']
baseFolder = '/home/sylfen/data_ext/'
FOLDERZIP = baseFolder + 'smallPower_zip_hourly/'
FOLDERPARKED_HOURS = baseFolder + '/smallPower_hourly/'

29 Mar 20:42:17 : plc_smallpower.pkl loaded in 1.89 ms 
---------------------------------------

FINISH LOADING CONFIGURATOR

beckhoff  :  1000.0 ms
beckhoff  :  100.0 ms
FINISH LOADING CONFIGURATOR

security check succeeded!

initialization of low pass tags done!



# Dumper collect and insert into database

In [ ]:
start = time.time()
quickTags = dumperSmallPower.dumpInterval['beckhoff'][0.1].argsAction[2]
slowTags = dumperSmallPower.dumpInterval['beckhoff'][1].argsAction[2]
beckhoffclient = dumperSmallPower.devices['beckhoff']
tags=quickTags
d=beckhoffclient.collectData(tags,'UTC')
df=pd.DataFrame(d).T;df
# c=beckhoffclient.insert_intodb(dumperSmallPower.dbParameters,'test_realtimedata',tags,'UTC')
# tag=tags[0];value,timestampz=d[tag]
# beckhoffclient.generate_sql_insert_tag(tag,value,timestampz,'test_realtimedata')

# Parker

### quick park on fly

In [4]:
dumperSmallPower.park_database(pool_tag=False)

29 Mar 20:43:27 : database for data <2022-03-29T20:43:27.583407+02:00 read in 153.39 ms


,tag,value
timestampz,,
2022-03-29 20:43:16.053960+02:00,SEH1.STB_GFC_01_PT_01.HM05,23.8288516998291
2022-03-29 20:43:16.053960+02:00,SEH1.STB_STK_03.ET_14.HM05,0.016384124755859375
2022-03-29 20:43:16.053960+02:00,SEH1.STB_STK_03.ET_15.HM05,0.02195453643798828
2022-03-29 20:43:16.053960+02:00,SEH1.STB_STK_03.ET_16.HM05,0.004588127136230469
2022-03-29 20:43:16.053960+02:00,SEH1.STB_STK_03.ET_17.HM05,0.010158538818359375
...,...,...
2022-03-29 20:43:27.536995+02:00,SEH1.STK_ALIM_01.ET_HM05,1.5499999523162842
2022-03-29 20:43:27.536995+02:00,SEH1.STK_ALIM_01.IT_HM05,0.0
2022-03-29 20:43:27.536995+02:00,SEH1.STK_ALIM_01.IT_HS00,0.0


### exportDatabase

In [ ]:
t1 = pd.Timestamp.now(tz='CET')
t1 = pd.Timestamp('2022-03-27 01:59:59',tz='CET')
t0 = pd.Timestamp('2022-03-26 00:00:00',tz='CET')
# t0 = t1-pd.Timedelta(hours=t1.hour,minutes=t1.minute,seconds=t1.second)
basename='-00-00-3-RealTimeData.csv'
dumperSmallPower.exportdb2zip(dumperSmallPower.dbParameters,t0,t1,FOLDERZIP,basename=basename)

In [ ]:
# pklfile=FOLDERZIP + (t0 + pd.Timedelta(days=1)).strftime(dbparker.streamer.format_dayFolder).split('/')[0]+basename.replace('.csv','.pkl')
pklfile=FOLDERZIP + '2022-03-25'+basename.replace('.csv','.pkl')
df = pd.read_pickle(pklfile)

In [ ]:
FOLDERPARKED_DAYS=baseFolder+'smallpower_daily_back/'
dbparker.streamer.park_DFday(df.reset_index(),FOLDERPARKED_DAYS,pool=False,showtag=True)

# alter parked data

## functions

In [ ]:
def read_csv_datetimeTZ(filename):
    start   = time.time()
    print("============================================")
    print('reading of file',filename)
    df = pd.read_csv(filename,parse_dates=['timestampz'],names=['tag','value','timestampz'])
    print('read in {:.2f} milliseconds'.format((time.time()-start)*1000))
    start = time.time()
    return df

def park_zip_hour_folder(filezip_hour):
    '''format of data in the zipFile should be 3 columns tag,value,timestampz with no header.'''
    start = time.time()
    #### unzip the file
    # try:
    with ZipFile(filezip_hour, 'r') as zipObj:
       zipObj.extractall(FOLDERZIP)
    ###read the file
    f_csv=filezip_hour.replace('.zip','.csv')
    df = read_csv_datetimeTZ(f_csv)
    ###remove the csv
    os.remove(f_csv)
    ###park the file
    streamer.park_DF_hour(df,FOLDERPARKED_HOURS,pool=False)
    message=filezip_hour+' parked in {:.2f} milliseconds'.format((time.time()-start)*1000)
    # except:
    #     print()
    #     print('************************************')
    #     message = filezip_hour+' failed to be parked'
    #     print(message)
    #     print('************************************')
    #     print()
    return message

def park_hourly2dayly(day,showtag=False):
    """ -day :'ex : 2022-02-15' """
    listhours=glob.glob(FOLDERPARKED_HOURS+day+'/*')
    listhours.sort()
    listTags = os.listdir(listhours[0])
    folderday=FOLDERPARKED_DAYS +'/'+ day+ '/'
    if not os.path.exists(folderday):os.mkdir(folderday)
    for tag in listTags:
        if showtag:print(tag)
        dfs=[pd.read_pickle(hour+'/' + tag) for hour in listhours]
        pd.concat(dfs).to_pickle(folderday + tag)
    return 

def applyCorrectFormat(d,cfg,newtz='CET',printag=False,debug=False):
    '''
    - format as pd.Series with name values and timestamp as index
    - remove index duplicates
    - convert timezone
    - apply correct datatype if bool,float or string
    '''
    print(d)
    tags = os.listdir(d)
    for t in tags:
        tagpath=d+'/'+t
        if printag:print(tagpath)
        try:
            df=pd.read_pickle(tagpath)
        except:
            print('pb loading',tagpath)
        if df.empty:
            print('dataframe empty for ',tagpath)
        else:
            ##### --- make them format pd.Series with name values and timestamp as index----
            if not isinstance(df,pd.core.series.Series):
                col_timestamp=[k for k in df.columns if 'timestamp' in k]
                df.set_index(col_timestamp)['value'].to_pickle(tagpath)
            ##### --- remove index duplicates ----
            df = df[~df.index.duplicated(keep='first')]
            ##### --- convert timezone----
            if isinstance(df.index.dtype,pd.DatetimeTZDtype):
                df.index = df.index.tz_convert(newtz)
            else:### for cases with changing DST at 31.10 or if it is a string
                df.index = [pd.Timestamp(k).astimezone(newtz) for k in df.index]
            #####----- apply correct datatype ------
            try:
                df = df.astype(cfg.dataTypes[cfg.dfplc.loc[t.strip('.pkl'),'DATATYPE']])
            except:
                print(t,' not in ',cfg.file_plc_xlsm)
            df.to_pickle(tagpath)

### raw data 

In [ ]:
folderPkl='/home/sylfen/data_ext/smallpower_daily_back/'
# folderPkl='/home/sylfen/data_ext/small/'
# folderPkl='/home/sylfen/data_ext/smallPower_daily/'
lf=os.listdir(FOLDERZIP)
lf=os.listdir(folderPkl)
lf.sort()
lf

In [ ]:
# df  = pd.read_pickle(FOLDERZIP+'/2022-03-25-00-00-x-RealTimeData.pkl')
dumperSmallPower.streamer.park_DFday(df.reset_index(),folderPkl,pool=False,showtag=True)

## modify data

In [ ]:
import smallPowerDash.smallPower as smallPower
cfg = smallPower.SmallPowerComputer(rebuildConf=False)
# listhoursfiles=glob.glob(FOLDERZIP+'/*2022-02-24*')
# listhoursfiles
# park_zip_hour_folder(listhoursfiles[0])
# with Pool(24) as p:p.map(park_zip_hour_folder,listhoursfiles)
# park_hourly2dayly('2022-02-24',showtag=True)
folderparked_days='/home/sylfen/data_ext/smallpower_daily_back/'
folderparked_days='/home/sylfen/data_ext/smallPower_daily/'
list_daysfiles=glob.glob(folderparked_days+'/*2022-03-22*')
list_daysfiles.sort()
for f in list_daysfiles: 
    applyCorrectFormat(f,cfg,newtz='UTC',printag=True,debug=False)

# functions comUtils

## setInterval test

In [ ]:
from time import sleep
import numpy as np
def sleepabit(initValue=0.9):
    t0 = dt.datetime.now().astimezone()
    print('start time task : ',t0.strftime('%H:%M:%S:%f')[:-4])
    value =initValue+np.random.randint(0,20)/100
    print('duration task : ',value)
    print('')
    sleep(value)

## initialize a M:S:000 pétante ! #####
now =dt.datetime.now().astimezone()
time.sleep(1-now.microsecond/1000000)
setTest=comUtils.SetInterval(1,sleepabit,0.99)
setTest.start()

## static compression test 

In [ ]:
# s=df.iloc[:,0]
# s2=streaming.staticCompressionTag(s,s.std(),method='reduce')

s=streaming.generateRampPlateau(nbpts=200)
precs=np.logspace(-2,0,6)*5
results = streaming.testCompareStaticCompression(s,precs)

## test reconnection

In [ ]:
dfplc=dumperSmallPower.dfPLC
tags = list(dfplc.index[dfplc['FREQUENCE_ECHANTILLONNAGE']==0.1])
quickNodes = {t:dumperSmallPower.nodesDict[t] for t in tags}
for k in range(2):
        # d=dumperSmallPower.collectData(quickNodes)
        c=dumperSmallPower.insert_intodb(quickNodes)
        time.sleep(0.2)

## check acquisition times

In [ ]:
dict2pdf = lambda d:pd.DataFrame.from_dict(d,orient='index').squeeze().sort_values()
s_collect = dict2pdf(dumperSmallPower.collectingTimes)
s_insert  = dict2pdf(dumperSmallPower.insertingTimes)

p = 1. * np.arange(len(s_collect))
## first x axis :
tr1 = go.Scatter(x=p,y=df,name='collectingTime',col=1,row=1)
## second axis
tr2 = go.Scatter(x=p,y=df,name='collectingTime',col=1,row=2)
title1='cumulative probability density '
title2='histogramm computing times '
# fig.update_layout(titles=)